In [5]:
import pandas as pd
import numpy as np
import nltk
import nltk.sentiment.util
import matplotlib.pyplot as plt
import pprint
import re
import gensim
import gensim.corpora as corpora
import os
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
from textblob import TextBlob

In [38]:
df = pd.read_csv("../../amazon_reviews_us_Grocery_v1_00.tsv", sep='\t', quoting=3)
df

marketplace  customer_id       review_id  product_id  product_parent  \
0                US     42521656  R26MV8D0KG6QI6  B000SAQCWC       159713740   
1                US     12049833  R1OF8GP57AQ1A0  B00509LVIQ       138680402   
2                US       107642  R3VDC1QB6MC4ZZ  B00KHXESLC       252021703   
3                US      6042304  R12FA3DCF8F9ER  B000F8JIIC       752728342   
4                US     18123821   RTWHVNV6X4CNJ  B004ZWR9RQ       552138758   
...             ...          ...             ...         ...             ...   
2402453          US     50385921   RZVFIWM72OSA0  B00004TBB0       738532654   
2402454          US     50377094  R35K50CF47311S  B00000K1X9       161697639   
2402455          US     50740327  R1N80O7XL5ODNC  B00000K1X9       161697639   
2402456          US     51700314  R1E9RSEX4R6TIR  B00000K1X9       161697639   
2402457          US     51207278  R3L60AR0DQYYMS  B00000K1X9       161697639   

                                             product_title product_category  \
0        The Cravings Place Chocolate Chunk Cookie Mix,...          Grocery   
1                  Mauna Loa Macadamias, 11 Ounce Packages          Grocery   
2        Organic Matcha Green Tea Powder - 100% Pure Ma...          Grocery   
3        15oz Raspberry Lyons Designer Dessert Syrup Sauce          Grocery   
4        Stride Spark Kinetic Fruit Sugar Free Gum, 14-...          Grocery   
...                                                    ...              ...   
2402453  Celestial Seasonings - Herbal Tea Caffeine Fre...          Grocery   
2402454                     479 NATL SHOW HRSE BAY PINTO 6          Grocery   
2402455                     479 NATL SHOW HRSE BAY PINTO 6          Grocery   
2402456                     479 NATL SHOW HRSE BAY PINTO 6          Grocery   
2402457                     479 NATL SHOW HRSE BAY PINTO 6          Grocery   

         star_rating  helpful_votes  total_votes vine verified_purchase  \
0                  5              0            0    N                 Y   
1                  5              0            0    N                 Y   
2                  5              0            0    N                 N   
3                  5              0            0    N                 Y   
4                  5              0            0    N                 Y   
...              ...            ...          ...  ...               ...   
2402453            4              0            4    N                 N   
2402454            5              0            0    N                 N   
2402455            5              5            5    N                 N   
2402456            5              1            1    N                 N   
2402457            5              0            0    N                 N   

                            review_headline  \
0        Using these for years - love them.   
1                                 Wonderful   
2                                Five Stars   
3                                Five Stars   
4                                Five Stars   
...                                     ...   
2402453                           Different   
2402454        This  Horse  Is  Gorgourges!   
2402455         brilliant, realistic model!   
2402456   don't have it yet..but  will soon   
2402457                               Great   

                                               review_body review_date  
0        As a family allergic to wheat, dairy, eggs, nu...  2015-08-31  
1        My favorite nut.  Creamy, crunchy, salty, and ...  2015-08-31  
2        This green tea tastes so good! My girlfriend l...  2015-08-31  
3        I love Melissa's brand but this is a great sec...  2015-08-31  
4                                                     good  2015-08-31  
...                                                    ...         ...  
2402453  The Amor Belhom Duo are likely the only musici...  2000-12-05  
2402454  Being  a  Breyer  collecter  almost  al

In [39]:
df = df.dropna()

In [40]:
polarity = []
subjectivity = []
for line in df['review_body']:
    line = line.replace('<br />', '')
    blob = TextBlob(line)
    pol = []
    sub = []
    for sentence in blob.sentences:
        pol.append(sentence.sentiment.polarity)
        sub.append(sentence.sentiment.subjectivity)
    polarity.append(np.array(pol).mean())
    subjectivity.append(np.array(sub).mean())
df['polarity'] = polarity
df['subjectivity'] = subjectivity

<ipython-input-40-d78fae4605f9>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['polarity'] = polarity
<ipython-input-40-d78fae4605f9>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['subjectivity'] = subjectivity


In [42]:
df.loc[(df['polarity'] < 0) & (df['subjectivity'] < 0.1)]['review_body']

135        we use these too. see previous reviews  also g...
490        Product arrived smashed up in a ball. Also, I ...
749        They arrived too sticky. We can't get the wrap...
865        after they sat for a few days they became chew...
1600                                 Don't waste your money.
                                 ...                        
2399768    I add this to my low-fat pina coladas.  A few ...
2400234    It saves one the trouble of putting sugar on r...
2400298    In order for tea to be labeled \\"decaff\\" is...
2402109    This is the smoothest coffee you will ever try...
2402335    I just spent the last 20 minutes trying to pur...
Name: review_body, Length: 6828, dtype: object